In [ ]:
db_path = r'D:\iliapl\topic_modeling\data\databases\bad_actors_Coronavirus_Project_POI_Followers_13-06-20.db'
author_stats_path = r'D:\iliapl\topic_modeling\data\output_data\author_stats'

In [ ]:
import json
import pandas as pd

# Load their tweets

with open(f'{author_stats_path}/during_pandemic_top40_percent_author_tweets_english.json', 'r') as f:
    during_pandemic_tweets = json.load(f)

during_pandemic_tweets = pd.DataFrame({'post_id': during_pandemic_tweets})

In [ ]:
import sqlite3 as sql

# Load authors using tweets

conn = sql.connect(db_path)
during_pandemic_tweet_authors = pd.read_sql('SELECT post_id, author FROM posts WHERE date >= date(\'2020-01-01\')', conn)
during_pandemic_authors = during_pandemic_tweet_authors[during_pandemic_tweet_authors['post_id'].isin(during_pandemic_tweets['post_id'])]['author']

In [ ]:
during_pandemic_authors = during_pandemic_authors.drop_duplicates()

In [ ]:
print(f'Number of authors with posts: {len(during_pandemic_authors)}')

In [ ]:
# load author table appropriate rows

authors_table = pd.read_sql('SELECT * FROM authors', conn)
authors_table = authors_table[authors_table['author_screen_name'].isin(during_pandemic_authors)]

In [ ]:
authors_table.columns

In [ ]:
import numpy as np

print('Friend count stats:')
print(f'Min. friend count: {min(authors_table["friends_count"])}')
print(f'Max. friend count: {max(authors_table["friends_count"])}')
print(f'Average friend count: {np.mean(authors_table["friends_count"])}')
print(f'Median friend count: {np.median(authors_table["friends_count"])}')
print(f'Std friend count: {np.std(authors_table["friends_count"])}')

In [ ]:
print('Follower count stats:')
print(f'Min. follower count: {min(authors_table["followers_count"])}')
print(f'Max. follower count: {max(authors_table["followers_count"])}')
print(f'Average follower count: {np.mean(authors_table["followers_count"])}')
print(f'Median follower count: {np.median(authors_table["followers_count"])}')
print(f'Std follower count: {np.std(authors_table["followers_count"])}')

In [ ]:
print('Status count stats:')
print(f'Min. status count: {min(authors_table["statuses_count"])}')
print(f'Max. status count: {max(authors_table["statuses_count"])}')
print(f'Average status count: {np.mean(authors_table["statuses_count"])}')
print(f'Median status count: {np.median(authors_table["statuses_count"])}')
print(f'Std status count: {np.std(authors_table["statuses_count"])}')

In [ ]:
authors_table['verified'].value_counts()

In [ ]:
import pandas as pd

labeled_authors_df = pd.read_csv(r'D:\iliapl\topic_modeling\data\databases\person_organization_classification\labeled_authors_V10.csv')
unlabeled_authors_df = pd.read_csv(r'D:\iliapl\topic_modeling\data\databases\person_organization_classification\all_unlabeled_predictions_using_description_and_SVM_classifier_inbalanced_to_label_V10.csv')

person_authors = set(labeled_authors_df[labeled_authors_df['author_sub_type'] == 'PERSON']['author_screen_name'])

# get X% top confidence authors

CONFIDENCE_PERCENTILE = 0.4

person_sorted_by_confidence = unlabeled_authors_df[unlabeled_authors_df['str_automatic_prediction'] == 'PERSON'].sort_values(by=['confidence_to_organization'], ascending=True)
max_organization_confidence = person_sorted_by_confidence.quantile(CONFIDENCE_PERCENTILE)[0]
percentile_person_authors = person_sorted_by_confidence[unlabeled_authors_df['confidence_to_organization'] < max_organization_confidence]

person_authors |= set(percentile_person_authors['author_screen_name'])

print(f'Minimum confidence for {len(person_authors)} authors: {1 - percentile_person_authors.iloc[-1]["confidence_to_organization"]}')


# get all authors with confidence > X%

#MIN_CONFIDENCE = 0.9

#person_authors |= set(unlabeled_authors_df[unlabeled_authors_df['confidence_to_organization'] < (1 - MIN_CONFIDENCE)]['author_screen_name'])

In [ ]:
person_authors = pd.DataFrame({'author': list(person_authors)})

In [ ]:
all_2019_posts = pd.read_sql('SELECT post_id, author, date FROM posts WHERE date < date(\'2020-01-01\') AND date >= date(\'2019-01-01\')', conn)

In [ ]:
all_author_2019_posts = all_2019_posts[all_2019_posts['author'].isin(person_authors['author'])]

In [ ]:
during_pandemic_tweets

In [ ]:
during_pandemic_authors

In [ ]:
authors_table[['author_guid', 'author_screen_name', 'author_osn_id']].reset_index(drop=True).to_csv('all_posted_authors_top40_percent.csv')